In [3]:
import pandas as pd
import time

def convert_hex(message):
    if not isinstance(message, float):
        return bytes.fromhex(message).decode(encoding="ISO-8859-1")
    
def convert_date(unixTS):
    return time.strftime("%m/%d/%Y %H:%M:%S", time.localtime(int(unixTS[:-3])))

revisions = pd.read_csv("../../revision.csv", converters={"date":convert_date,"message":convert_hex})

revisions.head(10)

,id,date,message
0,02 ce e1 52 38 36 73 a9 73 e2 2d 37 7b 6a 72 1...,07/13/2009 17:24:28,"Make stdin for test scripts empty, so that tes..."
1,04 8b 6e 52 83 41 31 f9 27 76 9a 9b 65 0c ce 5...,05/23/2017 17:18:40,Update CONTRIBUTING.md\n\nFixing broken issues...
2,04 ea d6 c5 8f 91 39 a7 9e 0c 0f d9 b5 bc 74 4...,01/20/2005 11:07:47,sane timestamps by default\n\ngit-svn-id: svn:...
3,05 51 aa a2 44 f5 7d 2d cb ec d8 de 51 79 e9 3...,04/07/2015 14:32:36,"Merge ""Document ports creating configuration f..."
4,05 c6 18 7f aa 4b c1 46 43 94 2f 76 34 a3 0f 8...,11/22/2013 17:31:50,Fixed #21497 -- Forced conversion to bytes for...
5,07 d6 f1 c4 3f fe cd 11 90 ad 9b bc e2 52 6d f...,07/18/2016 13:59:10,pin html5lib version in tox.ini\n
6,0d 1d a9 17 bd 4a a4 8f 19 85 4e 69 01 a6 20 d...,07/18/2014 20:27:58,Update notes for submitted changes\n\n* Add AP...
7,0d 78 49 03 f2 9e 34 e1 7e c9 64 2f 3c bc 99 5...,01/19/2015 10:55:52,Fix test @attr-ibutes in hadoop_test.py\n\nApp...
8,0f 4d 1d d7 67 82 4b 29 f8 bd b2 85 03 90 1b 1...,08/23/2017 14:19:34,Merge pull request #41662 from lordcirth/nfs-e...
9,0f 95 83 06 d8 55 a0 6e e4 48 6d 27 a9 61 b0 9...,10/23/2017 16:49:07,"Revert ""Update CONTRIBUTING.md""\n\nThis revert..."


In [4]:
for i in range(11):
    print(revisions.id[i] + "\n")
    print(revisions.message[i])
    print("*******************************************************************************************************")

02 ce e1 52 38 36 73 a9 73 e2 2d 37 7b 6a 72 1a 70 68 2a 71

Make stdin for test scripts empty, so that tests don't accidentally hang waiting
for stdin.


git-svn-id: https://llvm.org/svn/llvm-project/cfe/trunk@75506 91177308-0d34-0410-b5e6-96231b3b80d8

*******************************************************************************************************
04 8b 6e 52 83 41 31 f9 27 76 9a 9b 65 0c ce 5b 19 a0 80 ac

Update CONTRIBUTING.md

Fixing broken issues link.
*******************************************************************************************************
04 ea d6 c5 8f 91 39 a7 9e 0c 0f d9 b5 bc 74 4c 1e db 17 f6

sane timestamps by default

git-svn-id: svn://svn.twistedmatrix.com/svn/Twisted/trunk@12890 bbbe8e31-12d6-0310-92fd-ac37d47ddeeb

*******************************************************************************************************
05 51 aa a2 44 f5 7d 2d cb ec d8 de 51 79 e9 30 d6 72 4d 18

Merge "Document ports creating configuration for in-band inspection"
